# Inventory Management with Reinforcement Learning

This Jupyter notebook explores the application of Reinforcement Learning (RL) techniques in solving an inventory management problem. The goal is to train an agent to make optimal decisions regarding the quantity of products to order in a dynamic environment, balancing costs associated with overstocking and potential lost sales due to understocking.

The notebook is structured as follows:

1. **Environment Definition**
   - We start by defining a custom environment named `InventoryEnv` using the Gym library. This environment models the inventory management problem and provides a platform for the RL agent to interact with.

2. **Benchmark Policy**
   - We introduce a benchmark policy function that serves as a baseline strategy for comparison. This policy computes the order quantity based on a cost analysis of overage and underage.

3. **Reinforcement Learning Training**
   - We utilize the Proximal Policy Optimization (PPO) algorithm from Ray's RLlib library to train an agent. This involves setting up the PPO configuration, initializing the environment and trainer, and running the training loop.

4. **Loading and Testing Trained Model**
   - Once the agent is trained, we demonstrate how to load the trained model and evaluate its performance in the `InventoryEnv` environment.

# Create Environment

## Introduction
This Python code defines a custom environment called `InventoryEnv` using the Gym library. This environment models a simplified inventory management problem.

## Environment Details
1. **Initialization**:
    - The environment is initialized with several parameters, such as `lead time`, `storage_capacity`, `order_limit`, and others.
    - There are also maximum values defined for various quantities like `max_value`, `max_holding_cost`, etc.

2. **Observation Space**:
    - The state space is defined as a high-dimensional continuous space using the `spaces.Box` class from Gym.
    - It includes both inventory levels and various properties associated with the environment.

3. **Action Space**:
    - The action space is a one-dimensional continuous space, representing the quantity to be ordered. It ranges from 0 to 1, which will be mapped to an order quantity from 0 up to the order limit.

4. **State**:
    - The state of the environment is represented as an array, which includes information about inventory levels, prices, costs, and other parameters.

5. **Methods**:
    - `_normalize_obs`: This method normalizes the observations to a specific range, ensuring consistency in the state representation.

    - `reset`: Resets the environment to its initial state, including generating random values for parameters like price, cost, holding cost, etc.

    - `break_state`: Extracts specific components from the state for better readability and understanding.

    - `step`: Simulates a step in the environment. It takes an action (order quantity) as input and computes the resulting state, reward, and whether the episode is done.

## Step Function Details
1. **Order Placement**:
    - The action received is clipped to be between 0 and 1 and then mapped to an order quantity within the order limit.

2. **Inventory Update**:
    - The available capacity for inventory is calculated, and the actual quantity bought is determined based on this capacity.

3. **Demand Realization**:
    - The demand for the day is drawn from a Poisson distribution with mean `mu`.

4. **Reward Computation**:
    - The reward is computed based on various factors, including sales revenue, purchase cost, holding costs, and penalties for lost sales.

5. **State Update**:
    - The inventory levels are updated for the next day, and in-transit inventory is accounted for.

6. **Termination**:
    - If the maximum number of steps is reached, the episode is marked as done.

7. **Normalization of Reward**:
    - The reward is scaled down for numerical stability.

In [ ]:
"""
This code is modified from:
https://github.com/awslabs/or-rl-benchmarks/blob/master/News%20Vendor/src/news_vendor_environment.py
"""

import gym
import numpy as np
from gym import spaces
from scipy.stats import poisson


class InventoryEnv(gym.Env):
    def __init__(self, config={}):
        self.l = config.get("lead time", 5)
        self.storage_capacity = 4000
        self.order_limit = 1000
        self.step_count = 0
        self.max_steps = 40

        self.max_value = 100.0
        self.max_holding_cost = 5.0
        self.max_loss_goodwill = 10.0
        self.max_mean = 200

        self.inv_dim = max(1, self.l)
        space_low = self.inv_dim * [0]
        space_high = self.inv_dim * [self.storage_capacity]
        space_low += 5 * [0]
        space_high += [
            self.max_value,
            self.max_value,
            self.max_holding_cost,
            self.max_loss_goodwill,
            self.max_mean,
        ]
        self.observation_space = spaces.Box(
            low=np.array(space_low),
            high=np.array(space_high),
            dtype=np.float32
        )

        # Action is between 0 and 1, representing order quantity from
        # 0 up to the order limit.
        self.action_space = spaces.Box(
            low=np.array([0]),
            high=np.array([1]),
            dtype=np.float32
        )
        self.state = None
        self.reset()

    def _normalize_obs(self):
        obs = np.array(self.state)
        obs[:self.inv_dim] = obs[:self.inv_dim] / self.order_limit
        obs[self.inv_dim] = obs[self.inv_dim] / self.max_value
        obs[self.inv_dim + 1] = obs[self.inv_dim + 1] / self.max_value
        obs[self.inv_dim + 2] = obs[self.inv_dim + 2] / self.max_holding_cost
        obs[self.inv_dim + 3] = obs[self.inv_dim + 3] / self.max_loss_goodwill
        obs[self.inv_dim + 4] = obs[self.inv_dim + 4] / self.max_mean
        return obs

    def reset(self):
        self.step_count = 0

        price = np.random.rand() * self.max_value
        cost = np.random.rand() * price
        holding_cost = np.random.rand() * min(cost, self.max_holding_cost)
        loss_goodwill = np.random.rand() * self.max_loss_goodwill
        mean_demand = np.random.rand() * self.max_mean

        self.state = np.zeros(self.inv_dim + 5)
        self.state[self.inv_dim] = price
        self.state[self.inv_dim + 1] = cost
        self.state[self.inv_dim + 2] = holding_cost
        self.state[self.inv_dim + 3] = loss_goodwill
        self.state[self.inv_dim + 4] = mean_demand

        return self._normalize_obs()

    def break_state(self):
        inv_state = self.state[: self.inv_dim]
        p = self.state[self.inv_dim]
        c = self.state[self.inv_dim + 1]
        h = self.state[self.inv_dim + 2]
        k = self.state[self.inv_dim + 3]
        mu = self.state[self.inv_dim + 4]
        return inv_state, p, c, h, k, mu

    def step(self, action):
        beginning_inv_state, p, c, h, k, mu = \
            self.break_state()
        action = np.clip(action[0], 0, 1)
        action = int(action * self.order_limit)
        done = False

        available_capacity = self.storage_capacity \
                             - np.sum(beginning_inv_state)
        assert available_capacity >= 0
        buys = min(action, available_capacity)
        # If lead time is zero, immediately
        # increase the inventory
        if self.l == 0:
            self.state[0] += buys
        on_hand = self.state[0]
        demand_realization = np.random.poisson(mu)

        # Compute Reward
        sales = min(on_hand,
                    demand_realization)
        sales_revenue = p * sales
        overage = on_hand - sales
        underage = max(0, demand_realization
                          - on_hand)
        purchase_cost = c * buys
        holding = overage * h
        penalty_lost_sale = k * underage
        reward = sales_revenue \
                 - purchase_cost \
                 - holding \
                 - penalty_lost_sale

        # Day is over. Update the inventory
        # levels for the beginning of the next day
        # In-transit inventory levels shift to left
        self.state[0] = 0
        if self.inv_dim > 1:
            self.state[: self.inv_dim - 1] \
                = self.state[1: self.inv_dim]
        self.state[0] += overage
        # Add the recently bought inventory
        # if the lead time is positive
        if self.l > 0:
            self.state[self.l - 1] = buys
        self.step_count += 1
        if self.step_count >= self.max_steps:
            done = True

        # Normalize the reward
        reward = reward / 10000
        info = {
            "demand realization": demand_realization,
            "sales": sales,
            "underage": underage,
            "overage": overage,
        }
        return self._normalize_obs(), reward, done, info


### Benchmark Policy Function

#### `get_action_from_benchmark_policy(env)`
This function computes an action based on a simple benchmark policy. Here's detail for its steps:

1. **State Extraction**:
   - It first extracts specific components from the environment state using the `break_state` method. This includes inventory state, product price, product cost, holding cost, loss goodwill cost, and mean demand.

2. **Cost Computation**:
   - It calculates the cost of overage and the cost of underage based on the environment parameters `h` (holding cost), `p` (product price), `c` (product cost), and `k` (loss goodwill cost).

3. **Critical Ratio**:
   - The critical ratio is calculated. This ratio is used to determine the order quantity in order to balance the costs of overage and underage. It is the ratio of the cost of underage to the total cost (overage + underage).

4. **Horizon Target**:
   - The desired inventory level (`horizon_target`) is computed using the critical ratio and the mean demand `mu` for the given state.

5. **Deficit Calculation**:
   - The deficit is calculated as the difference between the desired inventory level and the current inventory level.

6. **Buy Action**:
   - The buy action is set to be the smaller of the deficit and the order limit defined in the environment.

7. **Action Normalization**:
   - The buy action is normalized to the range [0, 1] and returned as a list.

### Main Section

#### `if __name__ == "__main__":`
This block checks if the script is being run directly (not imported as a module).

#### Environment Initialization

- A random seed is set for reproducibility.
- An instance of `InventoryEnv` is created.

#### Episode Loop

The following actions are taken for each of the 2000 episodes:

1. **Episode Initialization**:
   - The episode counter is printed.
   - The environment is reset to its initial state, and the initial state is obtained.

2. **Episode Execution Loop**:
   - A loop runs until the episode is done.
   - In each iteration, the benchmark policy function is called to get an action.
   - The action is used to interact with the environment using `env.step(action)`.

   - The state, reward, and termination status are obtained.
   - The reward is appended to a list of episode rewards.

3. **Episode Summary**:
   - The total reward for the episode is computed as the sum of the rewards.
   - The average reward per time step is calculated.

4. **Printing Episode Statistics**:
   - Information about the current episode's rewards is printed.

In [ ]:
def get_action_from_benchmark_policy(env):
    inv_state, p, c, h, k, mu = env.break_state()
    cost_of_overage = h
    cost_of_underage = p - c + k
    critical_ratio = np.clip(
        0, 1, cost_of_underage
              / (cost_of_underage + cost_of_overage)
    )
    horizon_target = int(poisson.ppf(critical_ratio,
                         (len(inv_state) + 1) * mu))
    deficit = max(0, horizon_target - np.sum(inv_state))
    buy_action = min(deficit, env.order_limit)
    return [buy_action / env.order_limit]


if __name__ == "__main__":
    np.random.seed(100)
    env = InventoryEnv()
    episode_reward_avgs = []
    episode_total_rewards = []
    for i in range(2000):
        print(f"Episode: {i+1}")
        initial_state = env.reset()
        done = False
        ep_rewards = []
        while not done:
            # action = env.action_space.sample()
            action = get_action_from_benchmark_policy(env)
            # print("Action: ", action)
            state, reward, done, info = env.step(action)
            # print("State: ", state)
            ep_rewards.append(reward)
        total_reward = np.sum(ep_rewards)
        reward_per_day = np.mean(ep_rewards)
        # print(f"Total reward: {total_reward}")
        # print(f"Reward per time step: {reward_per_day}")
        episode_reward_avgs.append(reward_per_day)
        episode_total_rewards.append(total_reward)
        print(
            f"Average daily reward over {len(episode_reward_avgs)} "
            f"test episodes: {np.mean(episode_reward_avgs)}. "
            f"Average total epsisode reward: {np.mean(episode_total_rewards)}"
        )

# Training

### Ray and PPO Configuration

#### Import Statements
- The necessary libraries are imported, including NumPy for numerical operations, Ray for distributed computing, and the PPOTrainer from Ray's RLlib library.

#### Configuration
- A configuration dictionary (`config`) for PPO training is defined, which will be used to customize the training process.

- `num_gpus`: Specifies the number of GPUs to be used. In this case, it's set to 1, assuming a single GPU is available. Set to 0 if no GPU is available.

- `num_workers`: Determines the number of parallel workers used for collecting samples. This can be adjusted based on the available CPU cores.

- Several hyperparameters are set, influencing the learning process (e.g., learning rate, batch sizes, etc.). Two combinations (commented as "Combination 1" and "Combination 2") are provided, and one of them is chosen for training.

### Ray Initialization

- `ray.init()`: Initializes the Ray library, setting up the distributed computing environment for training.

### PPO Trainer Initialization

- `trainer = PPOTrainer(config=config, env=InventoryEnv)`: Initializes the PPO trainer using the provided configuration and the `InventoryEnv` environment.

### Training Loop

- The training loop is initiated with a `while True:` loop, indicating that training will continue indefinitely until manually interrupted.

- `result = trainer.train()`: This line performs a single training iteration using PPO. The resulting object `result` contains various metrics and information about the training process.

- `print(pretty_print(result))`: This prints the training results in a readable format.

- `mean_reward = result.get("episode_reward_mean", np.NINF)`: Extracts the mean episode reward from the training result.

- If the current mean reward is greater than the best mean reward seen so far (`best_mean_reward`), the model checkpoint is saved, and `best_mean_reward` is updated.

- The training loop continues, with the model improving over time.

In [ ]:
import numpy as np
import ray
from ray.tune.logger import pretty_print
from ray.rllib.agents.ppo.ppo import DEFAULT_CONFIG
from ray.rllib.agents.ppo.ppo import PPOTrainer

from inventory_env import InventoryEnv

# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     num_gpus = 1
#     print(f"Number GPU {num_gpus}")
# else:
#     num_gpus = 0
#     print(f"Number GPU {num_gpus}")


config = DEFAULT_CONFIG.copy()
config["env"] = InventoryEnv
config["num_gpus"] = 1  # Set this to 0 if you don't have a GPU.
config["num_workers"] = 4  # Set this based on the number of CPUs on your machine
# config["num_iterations"] = 5
# Combination 1
# config["clip_param"] = 0.3
# config["entropy_coeff"] = 0
# config["grad_clip"] = 0.01
# config["kl_target"] = 0.05
# config["lr"] = 0.0001
# config["num_sgd_iter"] = 10
# config["sgd_minibatch_size"] = 128
# config["train_batch_size"] = 10000
# config["use_gae"] = True
# config["vf_clip_param"] = 10
# config["vf_loss_coeff"] = 1
# config["vf_share_layers"] = True

# Combination 2
config["clip_param"] = 0.3
config["entropy_coeff"] = 0
config["grad_clip"] = None
config["kl_target"] = 0.005
config["lr"] = 0.001
config["num_sgd_iter"] = 5
config["sgd_minibatch_size"] = 8192
config["train_batch_size"] = 20000
config["use_gae"] = True
config["vf_clip_param"] = 10
config["vf_loss_coeff"] = 1
config["vf_share_layers"] = False

# For better gradient estimates in the later stages
# of the training, increase the batch sizes.
# config["sgd_minibatch_size"] = 8192 * 4
# config["train_batch_size"] = 20000 * 10

ray.init()
trainer = PPOTrainer(config=config, env=InventoryEnv)

# Use this when you want to continue from a checkpoint.
# trainer.restore(
#   "/home/enes/ray_results/PPO_InventoryEnv_2020-10-06_04-31-2945lwn1wg/checkpoint_737/checkpoint-737"
# )



best_mean_reward = np.NINF
while True:
    result = trainer.train()
    print(pretty_print(result))
    mean_reward = result.get("episode_reward_mean", np.NINF)
    if mean_reward > best_mean_reward:
        checkpoint = trainer.save()
        print("checkpoint saved at", checkpoint)
        best_mean_reward = mean_reward

# Evaluate

## Loading and Testing a Trained Model

The provided code is for loading a pre-trained model and testing it in the `InventoryEnv` environment. Let's break down what it does:

### Ray and PPO Configuration

- Similar to the previous code snippet, it starts with importing necessary libraries and setting up the PPO trainer's configuration using default settings. The environment is set to `InventoryEnv`.

### Ray Initialization

- `ray.init()`: Initializes Ray for distributed computing.

### Restoring a Checkpoint

- `trainer.restore()`: Restores a pre-trained model. You need to replace the path inside the parentheses with the actual checkpoint path. This is how you load a previously trained model to either continue training or to perform evaluation/testing.

### Main Section

- `if __name__ == "__main__":` checks if the script is being run directly.

- `np.random.seed(0)`: Sets a seed for reproducibility.

- `env = InventoryEnv()`: Creates an instance of the `InventoryEnv`.

- `episode_reward_avgs` and `episode_total_rewards` are empty lists that will be used to store rewards during testing.

### Testing Loop

- A loop is initiated for running a series of episodes (2000 in this case).

- For each episode:

    - `state = env.reset()`: Resets the environment and gets the initial state.
    
    - A loop is initiated to run a single episode until termination:
    
        - `action = trainer.compute_action(state)`: The trained policy network computes an action based on the current state.
        
        - `state, reward, done, info = env.step(action)`: The action is applied to the environment, and the resulting state, reward, and termination flag are obtained.
        
        - The reward is appended to `ep_rewards`.
        
    - Total rewards and average rewards per time step for the episode are computed and printed.

    - The average daily reward and average total episode reward over all episodes up to this point are calculated and printed.

In [ ]:
import numpy as np
import ray
from ray.rllib.agents.ppo.ppo import DEFAULT_CONFIG
from ray.rllib.agents.ppo.ppo import PPOTrainer

from inventory_env import InventoryEnv

config = DEFAULT_CONFIG.copy()
config["env"] = InventoryEnv

ray.init()
trainer = PPOTrainer(config=config, env=InventoryEnv)

trainer.restore(
    # Replace this with your checkpoint path.
    "ray_results/PPO_InventoryEnv_2023-09-26_14-57-438z9flexj/checkpoint_3300/checkpoint-3300"
)

if __name__ == "__main__":
    np.random.seed(0)
    env = InventoryEnv()
    episode_reward_avgs = []
    episode_total_rewards = []
    for i in range(2000):
        print(f"Episode: {i+1}")
        state = env.reset()
        done = False
        ep_rewards = []
        while not done:
            action = trainer.compute_action(state)
            state, reward, done, info = env.step(action)
            ep_rewards.append(reward)
        total_reward = np.sum(ep_rewards)
        reward_per_day = np.mean(ep_rewards)
        print(f"Total reward: {total_reward}")
        print(f"Reward per time step: {reward_per_day}")
        episode_reward_avgs.append(reward_per_day)
        episode_total_rewards.append(total_reward)
        print(
            f"Average daily reward over {len(episode_reward_avgs)} "
            f"test episodes: {np.mean(episode_reward_avgs)}. "
            f"Average total epsisode reward: {np.mean(episode_total_rewards)}"
        )